### 0. Imports and config

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from pubrecon.data import DataFrame, ImagesData
from pubrecon.model import RCNN

In [ ]:
data_path = "../data/in/"
work_path = "../data/out/"

### 1. DataFrame

In [ ]:
dataframe = DataFrame(data_path, pickle_path=os.path.join(work_path, 'dataframe.pickle'))

In [ ]:
dataframe.prepare_data(force_preparation=True, subsamples=10, verbose=1)

In [ ]:
dataframe.summary()

### 2. ImagesData

In [ ]:
imagesdata = ImagesData(dataframe, pickle_path=os.path.join(work_path, 'imagesdata.pickle'))

In [ ]:
# That part is quite long, beware!
imagesdata.prepare_images_and_labels(number_of_results=2500, iou_threshold=0.85, max_samples=15, verbose=1)

In [ ]:
imagesdata.summary()

### 3. Model

In [ ]:
model = RCNN(imagesdata, loss=None, opt=None, lr=0.001, verbose=2)

In [ ]:
model.train(epochs=2, split_size=0.10, checkpoint_path=os.path.join(work_path, 'checkpoint.h5'),
            early_stopping=True, verbose=2)

### 4. Metrics

In [ ]:
loss = model.history()['loss']
val_loss = model.history()['val_loss']
accuracy = model.history()['accuracy']
val_accuracy = model.history()['val_accuracy']

In [ ]:
# Loss
plt.plot(loss)
plt.plot(val_loss)
plt.title("Model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Loss", "Validation Loss"])
plt.show()

In [ ]:
print("Final loss: {}".format(loss)[-1])

In [ ]:
# Accuracy
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title("Model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy", "Validation Accuracy"])
plt.show()

In [ ]:
print("Final loss: {}".format(accuracy[-1]))

### 5. Recognition example

In [ ]:
imagesdata.show_image(6, show_infos=True, show_labels=True)

In [ ]:
model.test_image(6, show_infos=False, show_labels=False, number_of_results=2500, threshold=0.85, verbose=1)